In [ ]:
# Impotación de librerías
import pandas as pd
import numpy as np
from datetime import datetime
import re


In [ ]:
# Ingesta de datos del archivo (movies_dataset) dado por el cliente
df = pd.read_csv('Dataset/movies_dataset.csv')

********** ANÁLISIS EXPLORATORIO DE LOS DATOS **********

In [ ]:
df.head()
df.info()

In [ ]:
# Desanidar la columna belongs_to_collection
df['belongs_to_collection'] = df['belongs_to_collection'].fillna('{}') # Se rellena los NAN con diccionarios vacíos

df_collection = pd.json_normalize(df['belongs_to_collection'].map(eval)) # Normalizar datos estructurados en formato JSON en una tabla plana (DataFrame) donde cada clave del objeto JSON se convierte en una columna

df_collection = df_collection.drop(['poster_path', 'backdrop_path'], axis=1)
df_collection # Df nuevo de los datos desanidados de belongs_to_collection

In [ ]:
# Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0
df['revenue'] = df['revenue'].fillna(0)

In [ ]:
# Los valores nulos del campo release date deben eliminarse
df.dropna(subset=['release_date'], inplace=True)

In [ ]:
# Las fechas deberán tener el formato AAAA-mm-dd
# df['release_date'] = pd.to_datetime(df['release_date'], format='%B %d, %Y').dt.strftime('%Y-%m-%d')

# Convertir la columna "release_date" de object a datetime
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce') # Se reemplaza las fechas dejando de lado los errores

In [ ]:
# Crear la columna release_year donde se extraerá el año de la fecha de estreno.
df['release_year'] = df['release_date'].apply(lambda x: x.year if x is not pd.NaT else None)
df['release_year'] = df['release_year'].fillna(0).astype(int)

In [ ]:
# Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
# cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Reemplazar valores str por 0 en la columna budget
df['budget'] = df['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0)
df['budget'] = df['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0)
df['budget'] = df['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0)

# Cambiar formato de la columna budget de object a int
df['budget'] = df['budget'].astype(int)
df['return'] = df['revenue'] / df['budget']
# Reemplazar por 0 si el resultado es NaN (0/0)
df['return'].fillna(0, inplace=True)
# Reemplazar por 0 si el resultado es inf
df.loc[np.isinf(df['return']), 'return'] = 0

In [ ]:
# Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage
df.drop(['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], axis=1, inplace=True)

In [ ]:
def extraer_datos(df, columna):
    # Función para extraer los valores de una cadena, lista o diccionario
    def extraer_valores(valor):
        if isinstance(valor, str):
            # Si es una cadena, extraer los valores con la expresión regular
            pattern = r"'name': '([^']*)'"
            coincidencias = re.findall(pattern, valor)
            return coincidencias
        elif isinstance(valor, list):
            # Si es una lista, extraer los valores de cada elemento de la lista
            valores = []
            for elemento in valor:
                if 'name' in elemento:
                    valores.append(elemento['name'])
            return valores
        elif isinstance(valor, dict):
            # Si es un diccionario, extraer los valores de la clave 'name'
            if 'name' in valor:
                return valor['name']
        return None
    
    # Aplicar la función a la columna y sobrescribir la columna
    df['production_companies'] = df['production_companies'].apply(extraer_valores)
    #print(df['production_companies'])
    return df

# Aplicar la función para extraer los datos de la columna 'belongs_to_collection'
df_companies = extraer_datos(df, 'production_companies')

# Mostrar el resultado
df_companies.head()


In [ ]:
# Unir los 2 df's

df_concat = pd.concat([df_collection, df], axis=1)
#df_final = df_concat.drop_duplicates() # Eliminar los registros duplicasdos
df_final

In [ ]:
# Guardar el df en un csv llamado platforms.csv
# df_final.to_csv('Dataset\movies_final.csv', index=False)